<a href="https://colab.research.google.com/github/AndresPautrat/Topicos_UPC/blob/main/ortools_Bases1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import random
def generarDataset(i,t,d):
  dataset=[]
  for y in range(t):
    transaction=[]
    for x in range(i):
      if random.random()<=d:
        transaction+=[1]
      else:
        transaction+=[0]
    dataset+=[transaction]
  return dataset

In [ ]:
ni=5
nt=10
nd=0.05
dataset=generarDataset(ni,nt,nd)

In [ ]:
!pip install ortools

     |████████████████████████████████| 28.9MB 148kB/s 
     |████████████████████████████████| 1.3MB 49.4MB/s 
  Found existing installation: protobuf 3.10.0
    Uninstalling protobuf-3.10.0:
      Successfully uninstalled protobuf-3.10.0


In [ ]:
from ortools.sat.python import cp_model

In [ ]:
class VarArraySolutionPrinter(cp_model.CpSolverSolutionCallback):
    """Print intermediate solutions."""

    def __init__(self, variables):
        cp_model.CpSolverSolutionCallback.__init__(self)
        self.__variables = variables
        self.__solution_count = 0

    def on_solution_callback(self):
        self.__solution_count += 1
        for v in self.__variables:
            print('%s=%i' % (v, self.Value(v)), end=' ')
        print()

    def solution_count(self):
        return self.__solution_count

In [ ]:
model=cp_model.CpModel()
#variables and domains
items=[model.NewBoolVar("it"+str(i))for i in range(ni)]
trans=[model.NewBoolVar("tr"+str(i))for i in range(nt)]
freq=model.NewIntVar(0,nt,"freq")
print(items,trans)

[it0(0..1), it1(0..1), it2(0..1), it3(0..1), it4(0..1)] [tr0(0..1), tr1(0..1), tr2(0..1), tr3(0..1), tr4(0..1), tr5(0..1), tr6(0..1), tr7(0..1), tr8(0..1), tr9(0..1)]


In [ ]:
for t in range(nt):
  model.Add(sum([items[i]*(1-dataset[t][i]) for i in range(ni)])==0).OnlyEnforceIf(trans[t])
  model.Add(sum([items[i]*(1-dataset[t][i]) for i in range(ni)])>0).OnlyEnforceIf(trans[t].Not())
model.Add(freq==sum(trans))
model.Add(freq>=5)

solver=cp_model.CpSolver()
solution_printer=VarArraySolutionPrinter(items+[freq])
solver.SearchForAllSolutions(model,solution_printer)

print(solution_printer.solution_count(),"Itemsets")

it0=0 it1=0 it2=0 it3=0 it4=0 freq=10 
1 Itemsets
